# Moving onto temporal modelling.

**The way I want to do this is to make 2 models: One first-order Markov model for modelling the temporal relationships and then the DBN that will model the co-occurrences.**

In order to make the Markov model, we need the following steps:
1. define variables
2. discretize the variables (ie make 0/1 vars)
3. Estimate transition probabilities
4. Construct transition matrix
5. Combine transition matrices
6. Use the model for prediction

Of these, we already have 1. and 2., so we can move onto 3.

How to estimate **transition probabilities**: 
Count the number of transitions between states in dataset and divide by total number of transitions.



In [61]:
import os
import pandas as pd
import pprint 
import pickle
import numpy as np
pp = pprint.PrettyPrinter(indent=4)
from sklearn.model_selection import train_test_split

In [62]:
# load data

with open("facetouch_dataframes.pickle", 'rb') as f:
    ft_dataframes = pickle.load(f)

In [63]:
print(ft_dataframes['/home/roni/coding/mastersProject/src/csvOut/p_100/recording_3'].columns)

Index(['Unnamed: 0', 'participant', 'frame', 'pose', 'hand_left', 'hand_right',
       'leftHandTouching', 'rightHandTouching', ' face_id', ' timestamp',
       ' confidence', ' success', ' AU01_r', ' AU02_r', ' AU04_r', ' AU05_r',
       ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r',
       ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r',
       ' AU45_r', ' AU01_c', ' AU02_c', ' AU04_c', ' AU05_c', ' AU06_c',
       ' AU07_c', ' AU09_c', ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c',
       ' AU17_c', ' AU20_c', ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c',
       ' AU45_c', 'GAD Score', 'PHQ Score', 'combinedRightHand',
       'combinedLeftHand'],
      dtype='object')


In [64]:
print(len(ft_dataframes))

50


From the results of the correlation calculations we want to model **AU17, AU7 and AU14 for the left hand** and **AU14, AU12 and AU20 for the right hand** as those are the largest correlations.

Out of interest this correlates to the chin raiser, lid tightener, dimpler for left hand and dimpler, lip corner puller and lip stretcher for the right hand.

In [98]:
transition_cols = [' AU17_c', ' AU07_c',' AU14_c' , ' AU12_c',' AU20_c' , 'leftHandTouching', 'rightHandTouching' ]
transition_probs = { key:[0,0,0,0,0,0] for key in transition_cols } # [ 1 to 0, total 1, 0 to 1, total 0, 0 to 0 , 1 to 1]

Need to compute the probability P(t = 0| t-1 = 1) for each of these. We also need to split into train/test/val.

In [99]:
video_keys = list(ft_dataframes.keys())

# Split the keys into training, validation, and testing sets
train_keys, test_keys = train_test_split(video_keys, test_size=0.1, random_state=42)
train_keys, val_keys = train_test_split(train_keys, test_size=0.111, random_state=42)

# Create the training, validation, and testing sets as dictionaries
train_data = {key: ft_dataframes[key] for key in train_keys}
val_data = {key: ft_dataframes[key] for key in val_keys}
test_data = {key: ft_dataframes[key] for key in test_keys}

In [100]:
#ex = train_data['/home/roni/coding/mastersProject/src/csvOut/p_100/recording_3'][['leftHandTouching', 'rightHandTouching']]
#ex.to_csv('lefthanddiffsp100.csv')

df = pd.DataFrame({'column1': [False,False,True,True,False]})
d = df.astype(int).diff()
print(d)

   column1
0      NaN
1      0.0
2      1.0
3      0.0
4     -1.0


In [101]:
for df in train_data:
    for i in transition_cols:
        #if( True not in train_data[df][i].unique() ):
            #continue
        diffs = train_data[df][i].astype(int).diff()
        positive_transitions = (diffs == 1).sum() # this is 0 to 1
        negative_transitions = (diffs == -1).sum() # this is 1 to 0
        zero_to_zero = ((train_data[df][i] == 0) & (diffs == 0)).sum()
        one_to_one = ((train_data[df][i] == 1) & (diffs == 0)).sum()
        zeroes = (train_data[df][i] == 0).sum()
        ones = (train_data[df][i] == 1).sum()
        
        #print(i, positive_transitions, ones, negative_transitions, zeroes)
        transition_probs[i][0] += negative_transitions
        transition_probs[i][1] += ones
        transition_probs[i][2] += positive_transitions
        transition_probs[i][3] += zeroes
        transition_probs[i][5] += one_to_one
        transition_probs[i][4] += zero_to_zero
        
        
        
    

In [102]:
probabilities = {}

for i in transition_probs:
    t = transition_probs[i]
    print(i, t)
    probabilities[i] = [ t[0]/t[1], t[2]/t[3] , t[4]/t[3], t[5]/t[1] ]
    
pp.pprint(probabilities)

 AU17_c [8238, 197759, 8236, 333669, 325401, 189513]
 AU07_c [7126, 121126, 7134, 410302, 403142, 113986]
 AU14_c [5399, 131815, 5402, 399613, 394183, 126404]
 AU12_c [3636, 92489, 3635, 438939, 435270, 88847]
 AU20_c [6387, 168733, 6395, 362695, 356276, 162330]
leftHandTouching [1398, 5921, 1400, 525507, 524069, 4521]
rightHandTouching [303, 719, 304, 530709, 530366, 415]
{   ' AU07_c': [   0.05883129963839308,
                   0.017387192848194743,
                   0.9825494391935696,
                   0.9410531182405099],
    ' AU12_c': [   0.03931278314177902,
                   0.00828133294147934,
                   0.9916412075482015,
                   0.9606223442787791],
    ' AU14_c': [   0.040958919698061674,
                   0.013518078741182094,
                   0.9864118534682305,
                   0.9589500436217426],
    ' AU17_c': [   0.041656764041080305,
                   0.024683144073917564,
                   0.9752209525008316,
                   0.95

In [105]:
# now we make the transition matrix

# Define the states
states = [0, 1]
matrices = { key: np.zeros((len(states), len(states))) for key in transition_cols }

## Initialize the transition matrix with zeros
#transition_matrix = np.zeros((len(states), len(states)))
#print(matrices)

for m in matrices:
    transition_matrix = matrices[m]
    # Set the transition probabilities
    transition_matrix[0, 0] = probabilities[m][2] # 0 to 0
    transition_matrix[0, 1] = probabilities[m][1] # 0 to 1
    transition_matrix[1, 0] = probabilities[m][0] # 1 to 0
    transition_matrix[1, 1] = probabilities[m][3] # 1 to 1
    

    # Print the transition matrix
    print(transition_matrix)


[[0.97522095 0.02468314]
 [0.04165676 0.95830278]]
[[0.98254944 0.01738719]
 [0.0588313  0.94105312]]
[[0.98641185 0.01351808]
 [0.04095892 0.95895004]]
[[0.99164121 0.00828133]
 [0.03931278 0.96062234]]
[[0.98230193 0.01763189]
 [0.0378527  0.96205247]]
[[0.99726359 0.00266409]
 [0.23610877 0.76355345]]
[[9.99353695e-01 5.72818626e-04]
 [4.21418637e-01 5.77190542e-01]]


In [107]:
pp.pprint(matrices)

{   ' AU07_c': array([[0.98254944, 0.01738719],
       [0.0588313 , 0.94105312]]),
    ' AU12_c': array([[0.99164121, 0.00828133],
       [0.03931278, 0.96062234]]),
    ' AU14_c': array([[0.98641185, 0.01351808],
       [0.04095892, 0.95895004]]),
    ' AU17_c': array([[0.97522095, 0.02468314],
       [0.04165676, 0.95830278]]),
    ' AU20_c': array([[0.98230193, 0.01763189],
       [0.0378527 , 0.96205247]]),
    'leftHandTouching': array([[0.99726359, 0.00266409],
       [0.23610877, 0.76355345]]),
    'rightHandTouching': array([[9.99353695e-01, 5.72818626e-04],
       [4.21418637e-01, 5.77190542e-01]])}
